In [ ]:
from dotenv import load_dotenv
import pandas as pd
import os
from pinecone import Pinecone, ServerlessSpec
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
from tqdm import tqdm

pc = Pinecone(api_key="PINECONE_API_KEY")


In [48]:
load_dotenv()

True

In [13]:
collection = "korea-jd-test"

index_name = collection
if pc.has_index(index_name):
    print("인덱스가 이미 존재합니다")
    index = pc.Index(index_name)
    ## 데이터가 남아있을때 데이터 제거
    if len(index.describe_index_stats()["namespaces"]) > 0:
        index.delete(delete_all=True, namespace="")

In [14]:
## openAI의 embedding dimension과 동일
## dimension은 embedding model을 변경한다면 설정하기
pc.create_index(index_name, dimension=1536, metric="cosine",
    spec=ServerlessSpec(
    cloud="aws",
    region="us-east-1"
) ) #서버리스 인덱스 생성
index = pc.Index(index_name)

In [ ]:
## 인덱스 생성 확인
try:
    print("DB 상태 확인 중...")
    print(f"- 인덱스 정보: {index.describe_index_stats()}")
except Exception as e:
    print(f"DB 상태 확인 중 에러 발생: {str(e)}")
    raise

DB 상태 확인 중...
- 인덱스 정보: {'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}


# load data

In [21]:
data = pd.read_csv("/home/yhkim/code/JobPT/backend/get_similarity/data/korea_jd_sample_processed.csv")

In [ ]:
sentences = list(data["sentence"])

# chunking

In [31]:
sentences[0]

'"Empowering AI, Elice." 엘리스는 공공·기업 전 영역의 AI 전환을 이끌고, 모두가 AI 시대의 주역이 되도록 돕는 글로벌 AI 기업입니다. /*AI 인프라 혁신을 통한 AI 에코시스템 구축*/ • 최신 GPU(B200·H200) 및 국산 NPU를 탑재한 AI PMDC 통해 고성능 AI 인프라 구축 및 서비스 제공 • 국내 최고 수준의 전력 효율로 자체 구축한 AI 전용 데이터센터와 \'AI 인프라-플랫폼-모델-콘텐츠\' 통합 AI 생태계를 조성 • 정부 및 공공기관에 안정적인 클라우드 환경 제공이 가능한 보안인증(CSAP) 획득 /*한국어 특화 AI 모델로 교육 혁신을 선도*/ • AI SaaS 솔루션 \'엘리스LXP\'로 6,200여 기업·기관과 260만 명 학습자 지원 • 자체 개발한 한국어 특화 생성형 AI 모델 라이브러리로 목적별 맞춤 모델 제공 • 전국 초중고 AI 디지털교과서(AIDT)의 인프라·콘텐츠·플랫폼을 통합 공급 /*Product Div.*/ • 엘리스 플랫폼의 UI/UX를 설계하고, Web과 App을 개발 및 고도화합니다. • 우리의 주요 목표는 학생들에게 고품질의 온라인 교육 환경을 제공하기 위해 끊임없이 연구하고 개발하는 것입니다. 라이브 코딩, 라이브 스트리밍, 컨테이너 기반 실습 환경, AI 학습 도우미 등의 최신 기술을 적용합니다. • 단순 기능 개발이 아니라 ‘왜’라는 질문을 주고 받는 최고의 엔지니어/사이언티스트들로 이루어진 팀입니다. • 새로운 프로그래밍 과목(예. 확률론적 언어)을 배우고 지속적인 전문성 향상을 추구합니다. • 백엔드 개발자, 프론트엔드 개발자, 모바일 개발자, 프로덕트 매니저 및 프로덕트 디자이너와 같은 다양한 직무로 구성되어 있습니다. /*프론트엔드 엔지니어*/ • 혁신적인 AI 기반 교육, 평가 및 클라우드 플랫폼의 UI/UX를 개발하고 구현합니다. • 최신 기술 도입을 두려워하지 않는 팀 문화 가운데, 다양한 기술 스택들을 활용하여 글로벌 단위의 프로젝트에 참여할 기회를 가집니다

In [ ]:
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=100, separators=["#","##,""###","####","**","---","\r\n","\n\n", "\n", "\t", " ", ""])
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=100, separators=["<chunk_sep>","\r\n","\n\n", "\n", "\t", " ", ""])


In [51]:
sentences

['"Empowering AI, Elice." 엘리스는 공공·기업 전 영역의 AI 전환을 이끌고, 모두가 AI 시대의 주역이 되도록 돕는 글로벌 AI 기업입니다. /*AI 인프라 혁신을 통한 AI 에코시스템 구축*/ • 최신 GPU(B200·H200) 및 국산 NPU를 탑재한 AI PMDC 통해 고성능 AI 인프라 구축 및 서비스 제공 • 국내 최고 수준의 전력 효율로 자체 구축한 AI 전용 데이터센터와 \'AI 인프라-플랫폼-모델-콘텐츠\' 통합 AI 생태계를 조성 • 정부 및 공공기관에 안정적인 클라우드 환경 제공이 가능한 보안인증(CSAP) 획득 /*한국어 특화 AI 모델로 교육 혁신을 선도*/ • AI SaaS 솔루션 \'엘리스LXP\'로 6,200여 기업·기관과 260만 명 학습자 지원 • 자체 개발한 한국어 특화 생성형 AI 모델 라이브러리로 목적별 맞춤 모델 제공 • 전국 초중고 AI 디지털교과서(AIDT)의 인프라·콘텐츠·플랫폼을 통합 공급 /*Product Div.*/ • 엘리스 플랫폼의 UI/UX를 설계하고, Web과 App을 개발 및 고도화합니다. • 우리의 주요 목표는 학생들에게 고품질의 온라인 교육 환경을 제공하기 위해 끊임없이 연구하고 개발하는 것입니다. 라이브 코딩, 라이브 스트리밍, 컨테이너 기반 실습 환경, AI 학습 도우미 등의 최신 기술을 적용합니다. • 단순 기능 개발이 아니라 ‘왜’라는 질문을 주고 받는 최고의 엔지니어/사이언티스트들로 이루어진 팀입니다. • 새로운 프로그래밍 과목(예. 확률론적 언어)을 배우고 지속적인 전문성 향상을 추구합니다. • 백엔드 개발자, 프론트엔드 개발자, 모바일 개발자, 프로덕트 매니저 및 프로덕트 디자이너와 같은 다양한 직무로 구성되어 있습니다. /*프론트엔드 엔지니어*/ • 혁신적인 AI 기반 교육, 평가 및 클라우드 플랫폼의 UI/UX를 개발하고 구현합니다. • 최신 기술 도입을 두려워하지 않는 팀 문화 가운데, 다양한 기술 스택들을 활용하여 글로벌 단위의 프로젝트에 참여할 기회를 가집니

In [54]:
data.columns

Index(['url', 'job_category', 'job_name', 'title', 'company_name',
       'company_id', 'tag_name', 'tag_id', 'description', 'main_work',
       'qualification', 'preferences', 'welfare', 'tech_list', 'sentence'],
      dtype='object')

In [60]:
total_chunks = []
for i, desciption in enumerate(sentences):
    meta_data = [{"description":data.iloc[i]["description"],
    "job_url": data.iloc[i]["url"],
    "company": data.iloc[i]["company_name"]}]
    chunks = text_splitter.create_documents([desciption], meta_data)
    total_chunks.extend(chunks)

In [62]:
emb_model = OpenAIEmbeddings()

In [63]:
vector_store = PineconeVectorStore(index=index, embedding=emb_model)

In [67]:
total = len(total_chunks)
batch_size = 100

In [68]:
for start in tqdm(range(0, total, batch_size), desc="Upserting to Pinecone"):
    end = start + batch_size
    batch_docs = total_chunks[start:end]
    vector_store.add_documents(documents=batch_docs)

Upserting to Pinecone: 100%|██████████| 6/6 [00:26<00:00,  4.45s/it]


In [69]:
## 인덱스 생성 확인
try:
    print("DB 상태 확인 중...")
    print(f"- 인덱스 정보: {index.describe_index_stats()}")
except Exception as e:
    print(f"DB 상태 확인 중 에러 발생: {str(e)}")
    raise

DB 상태 확인 중...
- 인덱스 정보: {'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 512}},
 'total_vector_count': 512,
 'vector_type': 'dense'}
